GeekBrains, факультет Искусственного интелекта <br>
курс "Введение в обработку естественного языка" (старт 21 июня 2023)

### Урок 5. Part-of-Speech разметка, NER, извлечение отношений

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


In [5]:
!pip install pyconll

In [2]:
!mkdir datasets
# SynTagRus is now split into 3 parts. Download them all
!wget -O ./datasets/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./datasets/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./datasets/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
# Concatenate files together to get the full training set
!cat ./datasets/ru_syntagrus-ud-train-a.conllu ./datasets/ru_syntagrus-ud-train-b.conllu ./datasets/ru_syntagrus-ud-train-c.conllu > ./datasets/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

Џ®¤Ї ЇЄ  Ё«Ё д ©« datasets г¦Ґ бгйҐбвўгҐв.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [3]:
import pyconll
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [6]:
full_train = pyconll.load_from_file('data/ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('data/ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('data/ru_syntagrus-ud-train-c.conllu')

In [7]:
full_train.extend([*full_train_b, *full_train_c])

In [9]:
#full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('data/ru_syntagrus-ud-dev.conllu')

In [10]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [11]:
all_train_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[10:20]))
print('\n'.join(all_train_labels[10:20]))

Семен Еремеевич очень не любил , когда за этот стол кто-нибудь садился , и если видел отодвинутый стул , то всегда собственноручно подвигал его на место , так чтобы спинки образовывали ровную прямую линию .
В тот день , с которого начинается наш рассказ , жизнь в приемной протекала обычно .
Секретарша ставила сургучные печати на пакет , посетители ожидали своей очереди , радио играло сентиментальный вальс .
Ровно в десять часов раздался короткий звонок .
Быстрая секретарша на минуту скрылась в кабинете и проговорила , вернувшись : - Товарищ Ефимова , пожалуйста .
Молодая женщина в кокетливой шляпке прошла в кабинет .
Она , видимо , волновалась , потому что забыла закрыть за собой обе обитые черной клеенкой двери .
Семен Еремеевич медленно писал , склонив по привычке голову набок .
- Садитесь , пожалуйста , - сказал он , не поднимая глаз .
Ефимова села на краешек холодного кресла и посмотрела сквозь бронзовый забор чернильного прибора на Семена Еремеевича .
Семен Еремеевич очень не люби

In [12]:
test_sent = all_test_texts[15]
test_data = fdata_test

In [13]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(test_sent.lower().split()), unigram_tagger.evaluate(test_data))

[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8782863467673677

In [14]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

[('различные', None),
 ('определения', None),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', None),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.7101308678950452

In [15]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8839768214076438

In [16]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('различные', None),
 ('определения', None),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', None),
 ('или', None),
 ('неявной', None),
 ('форме', None),
 ('содержат', None),
 ('следующий', None),
 ('ряд', None),
 ('общих', None),
 ('требований', None),
 (':', None)]

0.4067191874470994

In [17]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8827137183410378

In [18]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8830522820496126

In [19]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [20]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [21]:
test_enc_labels = le.transform(test_label)

In [22]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char_wb', n_features=500)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [23]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

C:\Users\s2e\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [24]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8726234131662423

In [25]:
hvectorizer = HashingVectorizer(ngram_range=(1, 5), analyzer='char_wb', )
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [26]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

C:\Users\s2e\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [27]:
pred = lr.predict(X_test)

In [28]:
accuracy_score(test_enc_labels, pred)

0.9313419614535364

In [29]:
tvectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char_wb')
X_train = tvectorizer.fit_transform(train_tok)
X_test = tvectorizer.transform(test_tok)

In [30]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

C:\Users\s2e\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [31]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.9295377420552469

In [32]:
tvectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word')
X_train = tvectorizer.fit_transform(train_tok)
X_test = tvectorizer.transform(test_tok)

In [33]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

C:\Users\s2e\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [34]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.6531469624631178

Выводы:

1) UnigramTagger - 0.878  
2) BigramTagger - 0.710  
3) BigramTagger + UnigramTagger - 0.884  
4) TrigramTagger - 0.407  
5) TrigramTagger + BigramTagger + UnigramTagger - 0.883 (но чуть лучше, чем Trigram+Unigram)  
6) Свой (HashingVectorizer(char, 1-3, 500 feat) + LR 100 итераций) - 0.873  
7) Свой (HashingVectorizer(char, 1-5, max feat) + LR 100 итераций) - 0.931  
8) Свой (TfidfVectorizer(char, 1-3, max feat) + LR 100 итераций) - 0.93  
9) Свой (TfidfVectorizer(word, 1-1, max feat) + LR 100 итераций) - 0.661  

Общий вывод: наилучшие результаты даёт комбинация Unigram+Bigram и линейная регрессия с любым векторизатором без ограничения на кол-во фичей. Линейная регрессия работает при этом заметно медленнее, но даёт +5% точности. Попытки векторизировать данные не по символам, а по словам приводят к сильному ухудшению точности. Можно предположить, что это связано с тем, что при использовании символьных n-грам, модель может научиться понимать незнакомые слова по "похожести" на присутствующие в тренировочном наборе, в том числе по окончаниям и т.п., что даёт ей шанс верно угадать тэг для таких слов, в то время как модель, обучающаяся на словах, не имеет никаких данных о строение слова, и увидев новое слово не понимает, на что оно может быть похоже.